<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по его характеристикам

В связи с нехваткой свободного времени я сначала решила, что не успеваю разобраться с парсингом и взяла датасет, собранный Алексеем Титаренко в середине ноября. В итоге, с парсингом я немного разобралась, ноутбук будет на github, но для анализа все равно оставила чужой датасет.

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import re
import pandas_profiling
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.impute import SimpleImputer
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor,
GradientBoostingRegressor, BaggingRegressor, StackingRegressor)
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [2]:
#pip install lazypredict
#!pip install hyperopt

In [3]:
#from lazypredict.Supervised import LazyRegressor
#from hpsklearn import HyperoptEstimator
#from hpsklearn import any_regressor
#from hyperopt import tpe

In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [5]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [6]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [7]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [8]:
VERSION    = 16
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
DIR_TRAIN_EXTRA = '../input/parsed-avto/'
VAL_SIZE   = 0.20   # 20%

# Data

In [9]:
!ls '../input'

In [10]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
train_extra = pd.read_csv(DIR_TRAIN_EXTRA+'Parsed_avto.csv') # дополнительный датасет
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [11]:
pd.set_option('display.max_columns', None)

## Data Preprocessing

Перед объединением датасетов приведем данные к единому виду

In [12]:
# создадим список названий всех колонок
all_col = []
train_col = train.columns
train_extra_col = train_extra.columns
test_col = test.columns
for col in train_col:
    all_col.append(col)
for col in train_extra_col:
    if col not in all_col:
        all_col.append(col)
for col in test_col:
    if col not in all_col:
        all_col.append(col)
print(all_col)

In [13]:
train_missing = []
for col in all_col:
    if col not in train_col:
        train_missing.append(col)
print('В датасете train нет следующих колонок:', train_missing)

In [14]:
train_extra_missing = []
for col in all_col:
    if col not in train_extra_col:
        train_extra_missing.append(col)
print('В датасете train_extra нет следующих колонок:', train_extra_missing)

In [15]:
test_missing = []
for col in all_col:
    if col not in test_col:
        test_missing.append(col)
print('В датасете train нет следующих колонок:', test_missing)

Теперь посмотрим на сами данные:

In [16]:
train

In [17]:
train.info()

In [18]:
train_extra.head()

In [19]:
train_extra.info()

In [20]:
test.head(5)

In [21]:
test.info()

In [22]:
# Данной колонки нет в тренировочном датасете, посмотрим, какие значения она содержит
test.priceCurrency.unique()

In [23]:
train_extra.priceCurrency.unique()

### Что необходимо сделать:
- В датасете train:
  * Колонку Комплектация переименовать в complectation_dict, а колонку model в model_name
  * Убрать колонки hidden (пустая колонка), start_date (непонятно, что она обозначает и как ее обработать)
  * Добавить пустую колонку super_gen (чтобы не пропускать ее при объединении датасетов)
  * Добавить колонку vendor и заполнить ее значениями на основании модели машины
  * Убрать строки, где нет значения цены, т.к. они будут бесполезны для обучения модели
- В датасете train_extra:
  * Убрать колонки car_url, image (не выйдет обработать), equipment_dict (содержит информацию, которая уже есть в complectation_dict), model_info (пустая колонка), parsing_unixtime (но будем помнить, что данные были собраны в ноябре 2021 года), priceCurrency (данные собраны с сайта auto.ru и все цены указаны в российских рублях)
  * Убрать строки, в которых нет значения цены
- В датасете test:
  * Убрать колонки car_url, image (не выйдет обработать), equipment_dict (содержит информацию, которая уже есть в complectation_dict), model_info (содержит информацию, которая уже есть в колонке model_name), parsing_unixtime (но будем помнить, что данные были собраны в сентябре 2020 года), priceCurrency
  * В данном датасете отсутствует колонка price, т.к. это целевая переменная, но для объединения датасетов добавим ее и заполним нулями


#### Обработка train

In [24]:
# Переименуем колонки, которые отличаются только названием
train.rename(columns = {'Комплектация':'complectation_dict', 'model':'model_name'}, inplace=True)

In [25]:
# Уберем пустые колонки
train.drop(['hidden', 'start_date'], axis=1, inplace=True)

In [26]:
# Создадим колонку с пустыми значениями
train['super_gen'] = np.NaN

In [27]:
# посмотрим на колонку vendor в тестовой выборке
test['vendor'].unique()

In [28]:
# Создадим словарь брендов и производителей на основе тестовой выборки
vendor_dict = {k:v for v,k in test.groupby(['vendor', 'brand']).name.count().index}
train['vendor'] = train.brand.map(vendor_dict) # заполним столбец


In [29]:
train.vendor.value_counts(dropna=False)

In [30]:
# Уберем строки, где отсутствует цена, т.к. это тренировочная выборка
train.dropna(subset=['price'], inplace=True)

#### Обработка train_extra

In [31]:
# Удалим колонки, которые не понадобятся при обработке
train_extra.drop(['car_url', 'image', 'equipment_dict', 'model_info', 
                  'parsing_unixtime', 'priceCurrency'], axis=1, inplace=True)

In [32]:
# Удалим строки, где не указана цена, т.к. они будут бесполезны для обучения модели
train_extra.dropna(subset=['price'],inplace=True)

#### Обработка test

In [33]:
test.drop(['car_url', 'image', 'equipment_dict', 'model_info', 
                  'parsing_unixtime', 'priceCurrency'], axis=1, inplace=True)

In [34]:
# Создадим колонку с ценой, заполненную нулями
test['price'] = 0

#### Типы данных

In [35]:
# Посмотрим какой тип имею данные в трех датасетах
data_types = pd.DataFrame(columns=train.columns, index=['train', 'train_extra', 'test'])
for i in data_types.columns:
    data_types[i]['train'] = train[i].apply(type)[0]
    data_types[i]['train_extra'] = train_extra[i].apply(type)[0]
    data_types[i]['test'] = test[i].apply(type)[0]
data_types

Некоторые переменные имеют разные типы в разных датасетах:
- enginePower, Владельцы, Состояние, super_gen имеют тип float в train, str в train_extra и test
- complectation_dict имееет тип float в test, str в train и train_extra
- Таможня имеет тип bool в train, str в train_extra и test

### Объединение датасетов

In [36]:
# Разметим train и test 
train['train'] = 1
train_extra['train'] = 1 
test['train'] = 0

# Добавим колонку sell_id в тренировочные данные 
train['sell_id'] = 0  
train_extra['sell_id'] = 0 

# Обозначим где старые и новые данные
train['new_data'] = 0
train_extra['new_data'] = 1 
test['new_data'] = 0

In [37]:
# Объединяем датасеты для обработки 
combined_df = pd.concat([test, train, train_extra], join='inner', ignore_index=True)
combined_df.head()

In [38]:
combined_df.info()

In [ ]:
pandas_profiling.ProfileReport(combined_df)

### Предварительное заключение:
- 18 категориальных, 6 числовых и 2 булевых переменных. Тип данных в 3 столбцах не определен (это столбцы enginePower, Владельцы и Таможня, которые до объединения имели разные типы)
- Данные содержат мало дубликатов, но достаточно много пропусков. Особенно много пропусков в super_gen, vendor, Владение, Состояние
- Наблюдается сильная корреляция между productionDate и modelDate, modeDate и mileage а также vendor и brand (что логично)

## Очистка данных

#### bodyType

In [ ]:
combined_df.bodyType.unique()

Кроме типа кузова в данном столбце содержится информация о количестве дверей и названии модели, которые содержатся в других столбца. 

In [ ]:
# Оставим в столбце только тип кузова
combined_df.bodyType = combined_df.bodyType.apply(lambda x: x.lower().split()[0].strip() if isinstance(x, str) else x)
combined_df.bodyType.value_counts(normalize=True)

#### brand

In [ ]:
combined_df.brand.value_counts(dropna=False, normalize=True)

In [ ]:
# Посмотрим на список брендов, которые встречаются в тестовой выборке
combined_df[combined_df['train']==0].brand.value_counts()

In [ ]:
# заменим занчения, которых нет в тестовой выборке на np.NaN
combined_df['brand'] = combined_df.brand.apply(lambda x: x if x in ['BMW', 'VOLKSWAGEN','NISSAN','MERCEDES','TOYOTA','AUDI',
                                             'MITSUBISHI','SKODA','VOLVO','HONDA','INFINITI','LEXUS'] else np.NaN)
combined_df.brand.value_counts(dropna=False)

Придется удалить значительное число записей, видимо, при парсинге не было задано ограничение по брендам из тестовой выборки

In [ ]:
# удалим пропуски
combined_df.dropna(subset=['brand'], inplace=True)

#### color

In [ ]:
combined_df.color.value_counts()

In [ ]:
# некоторые цыета закодированы, создадим словарь с расшифровками
colors = {'040001': 'чёрный', 'FAFBFB': 'белый', 'CACECB':'серебристый', '97948F':'серый',
          '0000CC':'синий','EE1D19':'красный', '200204':'коричневый', '007F00':'зелёный', 
          'C49648':'бежевый', '22A0F8':'голубой', 'DEA522':'золотистый', '660099':'пурпурный',
          'FFD600':'жёлтый', '4A2197':'фиолетовый', 'FF8649':'оранжевый', 'FFC0CB':'розовый'}

In [ ]:
# Проведем замену
combined_df.color.replace(to_replace=colors, inplace=True)
combined_df.color.value_counts(normalize=True)

#### complectation_dict

In [ ]:
# оставим в колонке только количество перечисленных особенностей 
combined_df.complectation_dict = combined_df.complectation_dict.apply(lambda x: len(x.split('","')) if isinstance(x, str) else x)
combined_df.complectation_dict.value_counts()

#### description

In [ ]:
# оставим количество слов в описании
combined_df.description = combined_df.description.apply(lambda x: len(x) if isinstance(x, str) else x)
combined_df.description.value_counts()

#### engineDisplacement

In [ ]:
combined_df.engineDisplacement.unique()

In [ ]:
# создадим функцию, которая найдет значения среди этого хаоса
def find_float(value):
    extracted_value = re.findall('\d\.\d', str(value))
    if extracted_value:
        return float(extracted_value[0])
    return None

In [ ]:
combined_df.engineDisplacement = combined_df.engineDisplacement.apply(find_float)

In [ ]:
combined_df.engineDisplacement.value_counts(normalize=True, dropna=False)

#### enginePower

In [ ]:
combined_df.enginePower.unique()

In [ ]:
combined_df.enginePower = combined_df.enginePower.apply(lambda x: x.replace(" N12", "") if isinstance(x, str) else x)
combined_df.enginePower = combined_df.enginePower.replace('undefined', np.nan)
combined_df.enginePower = combined_df.enginePower.apply(lambda x: float(x))
combined_df.enginePower.value_counts(dropna=False)

engineDisplacement и enginePower отражают сходную информацию, но при этом engineDisplacement содержит 12% пропусков. Попытка создать словарь объем:мощность и заполнить на его основании пропуски engineDisplacement не принесла удовлетворительных результатов. Удалим столбец engineDisplacement, а в enginePower почистим пропуски.

In [ ]:
combined_df.drop(columns=['engineDisplacement'], inplace=True)

In [ ]:
combined_df.dropna(subset=['enginePower'], inplace=True)

#### fuelType

In [ ]:
combined_df.fuelType.value_counts(dropna=False)

#### mileage

In [ ]:
combined_df.mileage.isna().value_counts()

#### modelDate
Данная переменная имеет сильную корреляционную связь с переменной productionDate, т.к. productionDate лучше отражает возраст машины, оставим ее, а modelDate удалим (но позже, чтобы заполнить пропуски в productionDate, если такие имеются)

#### modelName

In [ ]:
combined_df.model_name.value_counts(dropna=False).head(30)

#### name

In [ ]:
combined_df.name

В разных строках находится разная по содержанию информация (в одних - объем двигателя и количество лошадиных сил, в других - название модели). Удалим данный признак.

In [ ]:
combined_df.drop(columns=['name'], inplace=True)

#### numberOfDoors

In [ ]:
combined_df.numberOfDoors.value_counts()

In [ ]:
# Посмотрим на записи, где указано, что дверей вообще нет
combined_df[combined_df.numberOfDoors == 0] 

Т.к. это автомобили с открытым верхом, можно не считать это ошибкой и оставить как есть

#### productionDate

In [ ]:
# проверим количество пропусков
sum(combined_df.productionDate.isna()==True)

Пропусков нет, можно удалять modelDate

In [ ]:
combined_df.drop(columns=['modelDate'], inplace=True)

#### vehicleConfiguration
Дублирует информацию из колонок vehicleTransmission, enginePower, numberOfDoors, поэтому удалим ее

In [ ]:
combined_df.drop(columns=['vehicleConfiguration'], inplace=True)

#### vehicleTransmission

In [ ]:
combined_df.vehicleTransmission.unique()

In [ ]:
# Заменим значения
combined_df.vehicleTransmission.replace({'роботизированная':'ROBOT', 
                                         'механическая':'MECHANICAL', 
                                         'автоматическая':'AUTOMATIC', 
                                         'вариатор':'VARIATOR'}, inplace=True)

In [ ]:
combined_df.vehicleTransmission.value_counts(dropna=False)

#### vendor

In [ ]:
combined_df.vendor.value_counts(dropna=False)

In [ ]:
# Посмотри на пропуски в данном столбце
combined_df[combined_df.vendor.isna()==True]

In [ ]:
# пропуски легко заплнить, исходя из бренда автомобиля
combined_df.vendor.fillna('EUROPEAN', inplace=True)

#### Владельцы ---> owners

In [ ]:
# переименуем колонку
combined_df.rename(columns={'Владельцы':'owners'}, inplace=True)

In [ ]:
combined_df.owners.value_counts(dropna=False)

In [ ]:
# оставим только числовые значения и преобразуем в тип float
combined_df.owners = combined_df.owners.apply(lambda x: x.replace('\xa0', ' ').split()[0] if isinstance (x, str) else x)
combined_df.owners = combined_df.owners.apply(lambda x: float(x))

In [ ]:
combined_df.owners.value_counts(dropna=False)

In [ ]:
# Посмотрим на пробег автомобилей, у которых отсутствует информация о количестве владельцев
combined_df[combined_df['owners'].isna()==True].mileage.value_counts()

In [ ]:
# Запоним количество владельцев единицами там, где пробег > 0.0, т.к. хотя бы один владелей был
combined_df.loc[(combined_df.owners.isna()) & (combined_df.mileage > 0.0), 'owners'] = 1.0

In [ ]:
# посмотрим к какому датасету принадлежали данные об автомобилях без пробега
combined_df[combined_df.mileage==0.0].train.value_counts()

Все новые автомобили (без пробега) находятся в тренировочной выборке, должно быть, данные про них были собраны по ошибке, т.к. по условию задания необходимо предсказывать стоимость подержанных автомобилей. Удалим эти записи.

In [ ]:
combined_df.dropna(subset=['owners'], inplace=True)
combined_df.owners.value_counts(dropna=False)

#### Владение
Пропуски составляют 68,7%, к тому же колонка не очень информативна, ведь есть количество владельцев и год производства. Удалим колонку.

In [ ]:
combined_df.drop(columns=['Владение'], inplace=True)

#### ПТС ---> vehicleLicence

In [ ]:
# переименуем колонку
combined_df.rename(columns={'ПТС':'vehicleLicence'}, inplace=True)

In [ ]:
combined_df.vehicleLicence.value_counts(dropna=False)

In [ ]:
# заменим значения и заполним пропуски модой
combined_df.vehicleLicence.replace({'Дубликат':'DUPLICATE', 
                                    'Оригинал':'ORIGINAL',
                                     np.NaN:'ORIGINAL'}, inplace=True)

In [ ]:
combined_df.vehicleLicence.value_counts(dropna=False)

#### Привод ---> gear

In [ ]:
# переименуем колонку
combined_df.rename(columns={'Привод':'gear'}, inplace=True)

In [ ]:
combined_df.gear.value_counts(dropna=False)

#### Руль ---> steeringWheel 

In [ ]:
# переименуем колонку
combined_df.rename(columns={'Руль':'steeringWheel'}, inplace=True)

In [ ]:
combined_df.steeringWheel.value_counts(dropna=False)

In [ ]:
# Заменим русские обозначения английскими
combined_df.steeringWheel.replace({'Левый':'LEFT', 
                                    'Правый':'RIGHT'}, inplace=True)

#### Состояние ---> condition

In [ ]:
# переименуем колонку
combined_df.rename(columns={'Состояние':'condition'}, inplace=True)

In [ ]:
combined_df.condition.value_counts(dropna=False)

In [ ]:
# Преобразуем колонку в бинарный тип
combined_df.condition = combined_df.condition.apply(lambda x: 1 if x == 'Не требует ремонта' else 0)

In [ ]:
combined_df.condition.value_counts(dropna=False)

#### Таможня ---> custom

In [ ]:
# переименуем колонку
combined_df.rename(columns={'Таможня':'custom'}, inplace=True)

In [ ]:
combined_df.custom.value_counts(dropna=False)

In [ ]:
# также преобразуем в бинарный тип
combined_df.custom = combined_df.custom.apply(lambda x: 1 if x == 'Растаможен' or x == True else 0)

In [ ]:
combined_df[combined_df['train']==0].custom.value_counts(dropna=False)

В тестовой выборке присутствуют только растаможеннные автомобили, а количество нерастаможенных в тренировочной ничтожно мало. Далее было выяснено, что данный признак не несет ценности для модели.

#### price, train, new_data

In [ ]:
sum(combined_df.price.isna())

In [ ]:
sum(combined_df.train.isna())

In [ ]:
sum(combined_df.new_data.isna())

Пропусков нет, все в порядке

### Удаление дубликатов

In [ ]:
combined_df.shape

In [ ]:
combined_df.drop_duplicates(inplace=True)
combined_df.shape

### Пропуски

In [ ]:
for column in combined_df.columns:
    misses = (1 - (combined_df[column].count() / combined_df.shape[0]))
    print('Процент пропусков в', column,': ', round(misses*100, 5), "%")

#### complectation_dict

In [ ]:
# т.к. мы оставили только количество, пустые значения можно заменить нулями
combined_df.complectation_dict.replace(np.NaN, 0, inplace=True)

#### description

In [ ]:
# аналогично, заполним нулями
combined_df.description.replace(np.NaN, 0, inplace=True)

#### super_gen

В данном столбце содержались не совсем одинаковые данные до объединения датасетов. 

Например в тестовом датасете: '{"id":"10373605","displacement":1197,"engine_type":"GASOLINE","gear_type":"FORWARD_CONTROL","transmission":"ROBOT","power":105,"power_kvt":77,"human_name":"1.2 AMT (105 л.с.)","acceleration":10.5,"clearance_min":155,"fuel_rate":5}'

А в датасете со свежими данными
"{'id': '2307782', 'name': '350', 'nameplate': '350', 'displacement': 3498, 'engine_type': 'GASOLINE', 'gear_type': 'REAR_DRIVE', 'transmission': 'AUTOMATIC', 'power': 272, 'power_kvt': 215, 'human_name': '350 3.5 AT (272 л.с.)'}"

Было бы полезно преобразовать этот столбец в несколько новых признаков (acceleration, clearence_min, fuel_rate) но эти данные можно извлечь только для тестового датасета.

Удалим данный столбец

In [ ]:
combined_df.drop(columns=['super_gen'], inplace=True)

### EDA

In [ ]:
combined_df.info()

In [ ]:
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 'gear', 'owners',
            'numberOfDoors', 'vendor', 'vehicleTransmission', 'vehicleLicence', 'steeringWheel']
num_cols = ['complectation_dict', 'enginePower', 'mileage', 'productionDate', 'description']
bin_cols = ['condition', 'custom']
help_cols = ['train', 'sell_id', 'new_data']
target_col = ['price']

#### Числовые переменые

In [ ]:
for col in num_cols:
    fig, ax = plt.subplots(1,2, figsize=(18,4))
    sns.distplot(combined_df[col], kde = False, rug=False, ax=ax[0], label='combined_df')
    sns.distplot(combined_df[combined_df.train==1][col], kde = False, rug=False, ax=ax[0], label='train')
    sns.distplot(combined_df[combined_df.train==0][col], kde = False, rug=False, ax=ax[0], label='test')
    sns.boxplot(combined_df[col], ax=ax[1])
    fig.legend()
    fig.show()

На графике распределений зеленым цветом обозначены тестовые данные, оранжевым - тренировочные данные, голубым - объединенные данные. Как видно на графиках распределения количественных переменных в тренировочной и тестовой выборках сходны. Однако имеется достаточно большое количество выбросов.

In [ ]:
combined_df[num_cols].describe()

In [ ]:
#посмотрим на влияние логарифмирования на распределение переменных
df = pd.DataFrame()
df['price'] = combined_df['price']

In [ ]:
for col in num_cols:
    df[col] = combined_df[col].apply(lambda w: np.log(w + 1))
    fig, ax = plt.subplots(1,2, figsize=(18,4))
    sns.distplot(df[col], kde = False, rug=False, ax=ax[0])
    sns.boxplot(df[col], ax=ax[1])
    fig.show()

После логарфмирование распределение признаков enginePower, description, mileage стало напоминать нормальное, но по прежнему остается проблема с выбросами. Столбец complectation_dict содержит много нулевых значений (и как следствие выбросов), он не принесет пользы модели, удалим его.

In [ ]:
# удалим колонку
combined_df.drop(columns=['complectation_dict'], inplace=True)

In [ ]:
# обновим список колонок
num_cols = ['enginePower', 'mileage', 'productionDate', 'description']

In [ ]:
# для оценки корреляции используем только тренировочные данные, где корректно указана цена
plt.figure(figsize=(16, 8))
correlation = combined_df[combined_df.train==1][num_cols + ['price']].corr()
sns.heatmap(correlation, annot = True, cmap = 'coolwarm')

productionDate и mileage имеют сильную отрицательную корреляционную связь, возможно, стоит их объединить в новый признак - пробег за год

In [ ]:
sns.pairplot(combined_df[num_cols])

In [ ]:
# Посмотрим, как соотносятся цены и год производства
sns.jointplot(x='productionDate', y='price', data=combined_df[combined_df.train==1], kind='reg', height=8)

Зависимость цены от года производства выглядит закономерно, за исключением нескольких сильно выбивающихся наблюдений. 

#### Категориальные переменные

In [ ]:
for col in cat_cols:
    if col not in ['model_name', 'name', 'brand']: # данные столбцы содержат слишком много уникальных значений
        fig, ax = plt.subplots(figsize=(18,6), ncols=2, nrows=1)
        combined_df[combined_df.train==1][col].value_counts().plot(kind='bar', ax=ax[0],color='b', label='train')
        combined_df[combined_df.train==0][col].value_counts().plot(kind='bar', ax=ax[0],color='c', label='test')
        sns.boxplot(x=col, y='price', data=combined_df[combined_df.train==1][combined_df['price'] <= combined_df['price'].quantile(0.9)], ax=ax[1])
        plt.xticks(rotation=90)
        fig.legend()
        fig.show()                                                                                            

Соотношение различных категорий признаков в тестовом и тренировочном датасетах сходно.

#### Бинарные переменные

In [ ]:
for col in bin_cols:
    fig, ax = plt.subplots(figsize=(18,6), ncols=2, nrows=1)
    combined_df[col].value_counts().plot(kind='bar', ax=ax[0])
    sns.boxplot(x=col, y='price', data=combined_df[combined_df.train==1][combined_df['price'] <= combined_df['price'].quantile(0.9)], ax=ax[1])
    plt.xticks(rotation=90)
    fig.show()                                                                                            

Автомобили, не требующие ремонта, в среднем дороже, что логично.
Переменная custom очень неравномерно представлена, кроме того ее значения почти не влияют на значение цены.

#### Целевая переменная - price

In [ ]:
combined_df.query('train == 1').price.hist();
plt.title('Распределение price', fontdict={'fontsize': 14});
plt.xlabel('price, RUB * 10^7');

In [ ]:
np.log2(combined_df.query('train == 1').price).hist();
plt.title('Распределение после логарифмирования', fontdict={'fontsize': 14});

Распределение целевой переменной после логарифмирование напоминает нормальное. Это значит, что для обучения модели лучше будет использовать логарифмированную переменную, т.к. моделям проще работать с переменными с нормальным распределением.

### Что необходимо сделать:
- Наблюдается сильная отрицательная корреляционная связь между переменными mileage и productionDate, целесоообразно содать дополнительный признак - пробег за год mileage_per_year
- 25-я перцентиль в распределении productionDate равна 2006 году,75-я - 2015 году, добавим переменную old для машин которые были произведены ранее 2006 года, и new - после 2015 года
- Создать колонку very_old для автомобилей, произведенных раньше 1970 года
- Создадим переменную top3_body_type для автомобилей с типом кузова седан, внедорожник и хэтчбек
- Переменная top_model для моделей, представленных в датасете более, чем 1500 машинами
- Переменная top5_color для автомобилей 5 самых популярных цветов
- Переменная top_engine_power для автомобилей с enginePower больше 400
- Переменная electro для электроавтомобилей и гибридных автомобилей
- Добавить столбцы - логарифм enginePower, mileage, description
- Откорректировать цену в тренировочной выборке с поправкой на курс доллара

### Feature ingineering

In [ ]:
# Добавим бинарные переменные на основании даты производства
combined_df['old'] = combined_df.productionDate.apply(lambda x: 1 if x < 2006 else 0)
combined_df['very_old'] = combined_df.productionDate.apply(lambda x: 1 if x < 1970 else 0)
combined_df['new'] = combined_df.productionDate.apply(lambda x: 1 if x > 2015 else 0)

In [ ]:
# Добавим бинарные перемнн
combined_df['top3_body_type'] = combined_df.bodyType.apply(lambda x: 1 if x in ['седан', 'внедорожник', 
                                                                                'хэтчбек'] else 0)
combined_df['top5_color'] = combined_df.color.apply(lambda x: 1 if x in ['чёрный','белый',
                                                                         'серебристый','серый',
                                                                         'синий'] else 0)
combined_df['top_engine_power'] = combined_df.enginePower.apply(lambda x: 1 if x>400 else 0)
combined_df['electro'] = combined_df.fuelType.apply(lambda x: 1 if x in ['гибрид','электро'] else 0)
combined_df['top_model'] = combined_df.model_name.apply(lambda x: 1 if x in['A6','OCTAVIA','5ER','FOCUS','X5','A4',
                                                                            'E_KLASSE','PASSAT','3ER','POLO','Camry',
                                                                            'OUTLANDER','X_TRAIL','SOLARIS','E-Класс',
                                                                            'LANCER','Passat','CAMRY','6','QASHQAI',
                                                                            'ASTRA','TIGUAN','5 серии'] else 0)

In [ ]:
combined_df['enginePower_log2'] = np.log2(combined_df.enginePower + 1)
combined_df['mileage_log2'] = np.log2(combined_df.mileage + 1)
combined_df['description_log2'] = np.log2(combined_df.description + 1)

Согласно информации с сайта banki.ru на 09.09.2020 курс доллара составлял 75.95 RUB за 1 USD, на 15.10.2021 - 71.78 RUB за 1 USD, т.е. на момент сбора train_extra курс составлял 0.945 от уровня сентября 2020 года
[https://www.banki.ru/products/currency/usd/]

После submission'а стало очевидно, что поправки на курс доллара недостаточно, чтобы нивелировать различия между значениями метрики МАРЕ в ноутбуке и на ЛБ. Эмпирическим методом было выявлено, что наиболее оптимальные значения на ЛБ получаются при применении коэффициента 0.75 к ценам 2021 года. Что свидетельствует о значительном росте спроса на автомобили

In [ ]:
combined_df.price = combined_df.apply(lambda row: row.price if row.new_data == 0 else row.price * 0.75, axis=1)

In [ ]:
combined_df.info()

In [ ]:
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'gear','model_name', 'owners', 'numberOfDoors', 'vendor', 'vehicleTransmission', 'vehicleLicence', 'steeringWheel']
new_num_cols = ['enginePower', 'mileage', 'productionDate', 'description', 'mileage_per_year', 'enginePower_log2', 'mileage_log2', 'description_log2']
new_bin_cols = ['condition', 'custom', 'old', 'very_old', 'new', 'top3_body_type', 'top5_color', 'electro', 'top_model', 'top_engine_power']
help_cols = ['train', 'sell_id', 'new_data']
target_col = ['price'] 

In [ ]:
imp_num = pd.Series(f_regression(
    combined_df[combined_df['train'] == 1][new_num_cols], combined_df[combined_df['train'] == 1]['price'])[0], index=new_num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')
plt.xlabel('Значимость переменных')

In [ ]:
cols_to_delete = ['enginePower_log2', 'mileage', 'description_log2']

#### Label Encoding

In [ ]:
for colum in cat_cols:
    combined_df[colum] = combined_df[colum].astype('category').cat.codes

In [ ]:
imp_cat = pd.Series(
    mutual_info_regression(
        combined_df[combined_df.train == 1][list(set(combined_df.columns) & set(cat_cols+new_bin_cols))], 
        combined_df[combined_df.train == 1]['price'], 
        discrete_features=True), index=list(set(combined_df.columns) & set(cat_cols+new_bin_cols))
)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')
plt.show()

In [ ]:
cols_to_delete = cols_to_delete+['very_old', 'custom']

In [ ]:
# так же добавим ненужные больше колонки 
cols_to_delete = cols_to_delete+['price', 'train', 'new_data']

Удаляю, в том числе колонку custom ввиду ее бесполезности (автомобили преимущественно растаможенные, экспериментально проверено, что наличие этой колонки никак не влияет на качество модели)

In [ ]:
X = combined_df.query('train == 1').drop(cols_to_delete, axis=1)
X_sub = combined_df.query('train == 0').drop(cols_to_delete, axis=1)
y = combined_df.query('train == 1').price

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Длинная история про ML
Для выбора оптимального алгоритма я собиралась использовать LazyRegressor, но ноутбук перезапускался из-за превышения объема памяти, поэтому пришлось экспериментрировать вручную.

После проверки предложенных в baseline моделей (наивной и catBoost) я решила начать с решающего дерева, прежде, чем переходить к ансамблям. Решающее дерево уже само по себе относительно неплохо сработало (МАРЕ около 23%), поэтому в качестве ансамбля я решила опробовать RandomForest (который в последствии оказался лучшей моделью).

Далее возникли проблемы с настройкой гиперпараметров, т.к. при запуске RandomizedSerchCV и GridSearchCV снова возникла проблема с перезапуском ноутбука. Код для уменьшения объема исользуемой памяти, найденный мной на StackOverFlow, проблему не решил. Было решено заниматься настройкой гиперпараметров только для тех моделей, которые с параметрами по умолчанию уже дают хороший результат. Пришлось завести дополнительный ноутбук в Юпитере для подбора гиперпараметров. НО: скачать train датасет с Kaggle не вышло, поэтому в Юпитере я тестировала модель только на train_extra, вероятно поэтому подобранные гиперпараметры не улучшали качество моделей на Kaggle.

В результате всех этих приключений и подбора корригирующего коэффициента для цен мне удалось получить значение метрики МАРЕ 13,58 на ЛБ после использования случайного леса с параметрами по умолчанию. На чем я и решила остановиться

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




In [ ]:
#tmp_train = X_train.copy()
#tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
#predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())

#оцениваем точность
#print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Точность по мерике МАРЕ 100%

# Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
#model = CatBoostRegressor(iterations = 5000,
#                          random_seed = RANDOM_SEED,
#                         eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE'],
#                          silent=True,
#                         )
#model.fit(X_train, y_train,
#         #cat_features=cat_features_ids,
#         eval_set=(X_test, y_test),
#         verbose_eval=0,
#         use_best_model=True,
#         #plot=True
#         )

In [ ]:
# оцениваем точность
#predict = model.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность по метрике МАРЕ 19.11%

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    

In [ ]:
#np.log(y_train)

In [ ]:
#model = CatBoostRegressor(iterations = 5000,
#                          random_seed = RANDOM_SEED,
#                          eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE'],
#                          silent=True,
#                         )
#model.fit(X_train, np.log(y_train),
#         #cat_features=cat_features_ids,
#         eval_set=(X_test, np.log(y_test)),
#         verbose_eval=0,
#         use_best_model=True,
#         #plot=True
#         )

In [ ]:
#predict_test = np.exp(model.predict(X_test))

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Как видим точность возросла до 15.39%

# lazypredict - LazyRegressor

Ноутбук перезапускается из-за превышения объема используемой памяти:( Придется перебирать все вручную 

In [ ]:
#reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
#models, predictions = reg.fit(X_train, X_test, y_train, y_test)
#print(models)

# DecisionTreeRegressor

In [ ]:
#model = DecisionTreeRegressor(random_state=RANDOM_SEED)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели 23.84%. Попробуем логарифмирование:

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели 23.38%

# RandomForestRegressor

In [ ]:
#model = RandomForestRegressor(random_state=RANDOM_SEED)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели 18.33%

In [ ]:
model = RandomForestRegressor(random_state=RANDOM_SEED)
model.fit(X_train, np.log(y_train))
y_pred = np.exp(model.predict(X_test))
y_sub = np.exp(model.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 15.84% на ЛБ 13.58%

In [ ]:
#n_estimators = [100, 150, 200, 250, 300, 350, 400]
#max_features = ['auto', 'sqrt']
#max_depth = [10, 15, 20, 25]
#max_depth.append(None)
#min_samples_split = [2, 4, 6, 8, 10]
#min_samples_leaf = [1, 2, 4]
#bootstrap = [True, False]
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}
#
#
#model_random = RandomizedSearchCV(estimator=model, param_distributions=random_grid, n_iter=10, 
#                               cv=3, verbose=2, random_state=42, n_jobs=-1)
#model_random.fit(X_train, y_train)

In [ ]:
#model_random.best_params_

In [ ]:
#model = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1,verbose=1, n_estimators=250,
#                             min_samples_split=8, min_samples_leaf=2, max_features='auto',
#                             max_depth=15, bootstrap=True)
#model.fit(X_train, y_train)

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 15.84%

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))
#y_sub = np.exp(model.predict(X_sub))
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

После настройки гиперпараметров точность модели составила 16,43%, на ЛБ 16.38%

# ExtraTreesRegressor

In [ ]:
#model = ExtraTreesRegressor(random_state=RANDOM_SEED)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 18.11%

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 15.92%

In [ ]:
#n_estimators = [100, 150, 200, 250, 300, 350, 400, 450, 500]
#max_features = ['auto', 'sqrt']
#max_depth = [10, 15, 20, 25]
#max_depth.append(None)
#min_samples_split = [2:10]
#min_samples_leaf = [1:4]
#bootstrap = [True, False]
#random_grid = {'n_estimators': n_estimators,
#               'max_features': max_features,
#               'max_depth': max_depth,
#               'min_samples_split': min_samples_split,
#               'min_samples_leaf': min_samples_leaf,
#               'bootstrap': bootstrap}
#
#model_random = RandomizedSearchCV(estimator=model, param_distributions=random_grid, n_iter=10, 
#                               cv=3, verbose=2, random_state=42, n_jobs=-1)
#model_random.fit(X_train, y_train)
#
#model_random.best_params_

In [ ]:
#model = ExtraTreesRegressor(random_state=RANDOM_SEED, 
#                            n_jobs=-1, 
#                            verbose=1, 
#                            n_estimators = 450, 
#                            min_samples_split = 5, 
#                            min_samples_leaf = 3, 
#                            max_features = 'auto', 
#                            max_depth = 15, 
#                            bootstrap = False)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Настройка гиперпараметров снова не дала результата. 20% с нелогарифмированной и 17% с логарифмированной ценой.

# AdaBoostRegressor

In [ ]:
#model = AdaBoostRegressor(random_state=RANDOM_SEED)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#180.86%

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#36.29%

# GradientBoosting

In [ ]:
#model = GradientBoostingRegressor(random_state=RANDOM_SEED)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [ ]:
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#33.22%

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#23.16%

# BaggingRegressor

In [ ]:
#model = BaggingRegressor(random_state=RANDOM_SEED)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#19.10#

In [ ]:
#model.fit(X_train, np.log(y_train))
#y_pred = np.exp(model.predict(X_test))
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#16.85%

# Стакинг

Объединим BaggingRegressor, ExtraTreesRegressor и RandomForestRegressor, которые самостоятельно давали хорошие результаты. В качестве мета-алгоритма используем LinearRegression

In [ ]:
#estimators = [
#    ('etr', ExtraTreesRegressor(random_state=RANDOM_SEED, n_jobs=-1, verbose=1)),
#    ('bagr', BaggingRegressor(random_state=RANDOM_SEED)),
#    ('cfr', RandomForestRegressor(random_state=RANDOM_SEED))
#             ]
#
#stack_log = StackingRegressor(estimators=estimators,
#                              final_estimator=LinearRegression()
#                              )
#
#stack_log.fit(X_train, np.log(y_train))
#
#y_pred = np.exp(stack_log.predict(X_test))
#
#print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred) * 100):0.2f}%.")

In [ ]:
#y_sub = np.exp(stack_log.predict(X_sub))

Точность модели 15.52%, на ЛБ 20.12%

# Submission

In [ ]:
sample_submission['price'] = y_sub
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)